### O que é um Banco de Dados Vetorial (Vector Store)?

Um **banco de dados vetorial** (também chamado de _vector store_) é uma estrutura ou serviço que permite armazenar e pesquisar dados na forma de **vetores de embeddings**. Em aplicações de NLP, esses vetores geralmente são gerados a partir de textos (ou consultas) por meio de um modelo de **embeddings**.

#### Por que usar Bancos de Dados Vetoriais?

- **Busca Semântica:** Em vez de pesquisar por palavras-chave, você compara vetores (embeddings) e recupera documentos mais relevantes com base na proximidade semântica.
- **Escalabilidade:** Muitos _vector stores_ (como Chroma, FAISS, Milvus, Pinecone) são otimizados para lidar com milhões de vetores e executar buscas em tempo satisfatório.
- **Operações Avançadas:** Além da busca por similaridade, alguns bancos suportam filtros por metadados, reordenação de resultados para diversidade (Maximal Marginal Relevance), pesquisa híbrida (keyword + similaridade), etc.

Quando utilizamos um banco de dados vetorial, o **fluxo** geral é:

1. **Carregar e dividir textos** (opcional, mas comum em caso de documentos extensos).
2. **Gerar embeddings** para cada pedaço de texto.
3. **Armazenar** esses embeddings (e seus metadados) no _vector store_.
4. **Consultar** o _vector store_ a partir de um texto ou de um embedding para recuperar os documentos mais relevantes de maneira semântica.

---

##### Exemplo Prático de Criação e Consulta de um Banco de Dados Vetorial (Chroma db)

[Documentação](https://python.langchain.com/docs/integrations/vectorstores/chroma/)

Considere o texto:
 

Vamos criar um documento dele como se fosse feito obtido via `document loaders`:

In [ ]:
%run ../helpers/00-llm.ipynb

In [ ]:
text="""A inteligência artificial (IA) é uma área da ciência da computação que tem revolucionado diversas indústrias e aspectos da vida cotidiana. Mas, o que realmente significa "inteligência artificial"? Trata-se de sistemas computacionais capazes de realizar tarefas que, anteriormente, só poderiam ser executadas por seres humanos, como reconhecimento de fala, tomada de decisão e aprendizado com dados. Impressionante, não é? Esses sistemas utilizam algoritmos avançados e grandes volumes de dados para identificar padrões, adaptarem-se a novas situações e fornecerem soluções inovadoras.

Um dos maiores avanços recentes em IA é o aprendizado profundo (ou deep learning). Essa técnica permite que máquinas realizem tarefas extremamente complexas, como diagnosticar doenças a partir de imagens médicas ou até mesmo compor músicas! Curioso como isso funciona? Redes neurais artificiais – inspiradas no funcionamento do cérebro humano – processam informações em múltiplas camadas, identificando nuances que seriam impossíveis para métodos tradicionais. Como resultado, a IA tem transformado áreas como saúde, finanças e transporte, promovendo eficiência e inovação em escala global.

No entanto, a expansão da inteligência artificial também levanta questões importantes. Estamos preparados para lidar com os desafios éticos que a IA traz? Por exemplo: como garantir que algoritmos de IA sejam imparciais e inclusivos? Além disso, há preocupações sobre o impacto no mercado de trabalho – algumas profissões podem ser substituídas por máquinas. Apesar desses desafios, uma coisa é certa: a inteligência artificial já não é mais uma tecnologia do futuro; é uma realidade do presente, moldando o mundo ao nosso redor com potencial ilimitado!"""
 

In [ ]:


from langchain_core.documents import Document

texto_original = Document(page_content=text)

# Vamos criar agora uma lista de documentos:
docs = [texto_original]


#### Passo 1: Preparar o Ambiente

**Instalar ou configurar as dependências** que serão utilizadas. Por exemplo, se formos usar o banco vetorial **Chroma** (local) e o modelo de embeddings da OpenAI, podemos fazer:

Caso use outro _vector store_, como FAISS, Pinecone, Milvus, etc., instale a biblioteca correspondente.

#### Passo 2: Inicializar o Modelo de Embeddings

Como todo _vector store_ depende de vetores de embeddings, vamos configurar a **chave de API** da OpenAI (caso ainda não esteja definida) e criar o nosso objeto de embeddings:

In [ ]:
from helpers.llm import initialize_llm, logger, pretty_print
 
llm, _, embeddings = initialize_llm()


#### Passo 3: Carregar e (Opcionalmente) Dividir Documentos

Para exemplificar, vamos carregar um arquivo de texto (`state_of_the_union.txt`). Em seguida, dividiremos esse texto em “chunks” menores, pois para a busca semântica normalmente trabalhamos com pedaços de texto menores.

In [ ]:
from langchain_text_splitters import CharacterTextSplitter


# Divide o documento em partes menores (chunks) de tamanho 1000 caracteres
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documentos = text_splitter.split_documents(docs)

print(documentos)

#### Passo 4: Criar o Banco Vetorial (Exemplo: Chroma)

Agora, utilizamos o `Chroma` (um _vector store_ que roda localmente) para armazenar os embeddings dos documentos:

In [ ]:
from langchain_chroma import Chroma

# Cria o banco de dados vetorial, gerando os embeddings dos documentos
Chroma.from_documents(documentos, collection_name="nome_colecao", embedding=embeddings, persist_directory="./meu_banco")

Sob o capô, cada chunk de texto de `documents` é convertido em um vetor de floats (pelo modelo de embeddings) e armazenado no banco, junto com alguma forma de identificação e metadados.

#### Passo 5: Realizar Buscas Semânticas

Depois de criado, podemos consultar o _vector store_ usando, por exemplo, o método `.similarity_search(query)` para buscar documentos que respondam a uma pergunta ou se relacionem a um tópico específico.

Logo, primeiro conectar ao banco:

In [ ]:
db = Chroma(  
    persist_directory="./meu_banco",  
    collection_name="nome_colecao",  
    embedding_function=embeddings,  
)

In [ ]:
query = "Na expansão da inteligência artificial quais questões importantes são levantadas?"
pedacoes_retornados = db.similarity_search(query, k=2)

# Total de docs retornados -> O default de similarity_search é K=3, mas escolhemos k= 2
print(len(pedacoes_retornados))
# Exibir o conteúdo do primeiro documento retornado
print(pedacoes_retornados[1].page_content)

O processo interno:

1. Gera embedding para a `query`.
2. Compara esse embedding com cada um dos embeddings armazenados.
3. Retorna os documentos mais semanticamente similares.

---

#### Interface

Para facilitar o uso e padronizar a interação, a LangChain oferece uma **interface de vector store** que define métodos básicos:

- **`add_documents(documents)`:**  
    Adiciona documentos (lista de `Document`) ao banco vetorial.
    
    - `documents`: lista de objetos `Document`, cada um contendo o texto (`page_content`) e metadados.
    - `ids`: lista opcional com identificadores únicos para cada documento.
- **`delete_documents(ids=[...])`:**  
    Remove documentos específicos, identificados por seus IDs.
    
- **`similarity_search(query, k=4, filter=None)`:**  
    Faz a busca semântica, retornando os `k` documentos mais similares ao `query`.
    
    - `query`: texto que será transformado em embedding.
    - `k`: quantidade de documentos mais similares a retornar.
    - `filter`: dicionário ou estrutura de filtragem baseado em metadados, se suportado pelo vector store.
- **`similarity_search_by_vector(embedding_vector, k=4, filter=None)`:**  
    Variante de busca que recebe diretamente um vetor de embedding ao invés de uma string.
    

Cada implementação de vector store (Chroma, FAISS, Pinecone, Qdrant etc.) segue a mesma estrutura de interface, porém pode oferecer recursos adicionais como re-indexação, reordenação avançada, busca híbrida, entre outros.


#### Outras Funcionalidades Importantes

##### 1. Similaridade por Vetor Direto

Em vez de passar uma string, podemos gerar manualmente o embedding e chamar `.similarity_search_by_vector(embedding_vector)`:

In [ ]:
embedding_vector = embeddings.embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

Isso é útil se você já tiver o embedding calculado ou se quiser maior flexibilidade de manipular os vetores antes de pesquisar.

##### 2. Operações Assíncronas

Em cenários de alta carga ou quando seu _vector store_ está hospedado em um serviço externo, chamadas assíncronas podem trazer ganho de performance. A LangChain fornece métodos com `a` de assíncrono, como `asimilarity_search`. Por exemplo:

In [ ]:
docs = await db.asimilarity_search(query)
print(docs[0].page_content)

Isso permite que seu código não fique bloqueado enquanto aguarda a resposta do serviço.

##### 3. Exclusão e Atualização de Documentos

Alguns _vector stores_ permitem remover ou atualizar documentos. Na interface padrão, temos métodos como `.delete_documents(ids=[])`, para remover documentos com base em seus IDs. Isso é útil quando um documento precisa ser retirado do seu índice, ou se deseja fazer uma substituição de conteúdo.

##### 4. Parâmetros Avançados de Busca

- **k**: Quantidade de documentos mais similares a retornar.
- **filter**: Possibilidade de filtrar por metadados. Por exemplo, retornar apenas documentos cujo campo `source` tenha valor `"tweet"`.

Exemplo de uso (Pinecone ou outros que suportem filtros de metadados):

Imagine que no banco há vários tweets diferentes e postagens do threads. Para filtrar da coleção apenas os dados de tweet, poderíamos usar o parâmetro de filtro: `filter`.

In [ ]:
# Imagine que no banco há vários tweets diferentes e postagens do threads. Para filtrar da coleção apenas os dados de tweet, poderíamos usar o parâmetro de filtro: `filter`.

docs = db.similarity_search(
    query="Quais tweet ",
    k=2,
    filter={"source": "tweet"}
)

Essas opções variam conforme o _vector store_, então verifique a documentação específica.

---

### Conceitos Fundamentais

##### Métricas de Similaridade

Para comparar vetores, são utilizadas funções matemáticas que avaliam a “distância” ou “semelhança” entre dois vetores:

- **Similaridade de Cosseno (Cosine Similarity):** Mede o cosseno do ângulo entre dois vetores.
- **Distância Euclidiana:** Mede a distância em linha reta entre dois pontos/vetores.
- **Dot Product (Produto Interno):** Mede a projeção de um vetor em outro.

A decisão sobre qual métrica usar depende do modelo de embeddings e do comportamento desejado. Frequentemente, “cosine similarity” é usada para lidar com texto.

##### Algoritmos de Indexação

Para evitar comparar exaustivamente o embedding de cada documento com o embedding da query, a maior parte dos _vector stores_ adota estruturas de indexação como **HNSW** ou **IVF** (no caso do FAISS), que permitem buscas aproximadas e muito mais rápidas em grandes conjuntos de vetores.

---

##### Metadados e Filtros

Quando inserimos documentos, podemos armazenar junto metadados como autor, data, fonte, etc. Na hora da busca, alguns bancos vetoriais oferecem filtros para recortar apenas documentos que batam com determinado critério de metadado. Isso possibilita **buscar por contexto** (por exemplo, “só me mostre documentos da categoria X ou do autor Y”).

---

##### Técnicas Avançadas

1. **Hybrid Search (Busca Híbrida):** Combina busca por similaridade semântica (via embeddings) com busca por palavra-chave (keyword). Ajuda a conciliar os dois mundos: correspondências exatas (palavras-chave) e correspondência semântica.
    
2. **Maximal Marginal Relevance (MMR):** Serve para reordenar resultados para aumentar a diversidade das respostas, evitando que todos os resultados sejam muito parecidos entre si.
    
3. **Re-rankers e Transformers:** Em cenários mais complexos, pode-se aplicar uma segunda etapa de reordenação, usando um modelo transformer para verificar a relevância de cada candidato.
    

---

##### Conclusão

Bancos de dados vetoriais (**vector stores**) são parte essencial de aplicações avançadas de NLP e IA generativa. Eles permitem armazenar e recuperar documentos com base em **similaridade semântica**, o que torna as buscas muito mais poderosas do que buscas baseadas em palavras-chave.

**Passo a passo resumido:**

1. **Carregar os documentos** e, opcionalmente, dividi-los em chunks menores.
2. **Gerar embeddings** para cada chunk com um modelo, como o da OpenAI.
3. **Armazenar** esses vetores e metadados em um _vector store_ (Chroma, FAISS, Pinecone etc.).
4. **Consultar** esses dados via `.similarity_search` (inserindo texto) ou `.similarity_search_by_vector` (inserindo vetores).
5. **Filtrar ou refinar** resultados usando técnicas avançadas como metadados, MMR, busca híbrida, etc.

Com essa base, você já está preparado para criar e consultar bancos de dados vetoriais para múltiplas aplicações: motores de busca semântica, chatbots contextuais, recomendação de conteúdo, análise de similaridade, e muito mais!

##### Links Auxiliares:

1 - Bancos vetoriais suportados pelo LangChain: https://python.langchain.com/docs/integrations/vectorstores/

In [ ]:
import os  
from langchain_community.document_loaders import TextLoader  
 
from langchain_text_splitters import CharacterTextSplitter  
from langchain_chroma import Chroma  
  
  
# Função: Divide o documento em partes menores (chunks) de tamanho 1000 caracteres, com prioriade para a quebra por  
# paragrafo.  
def divide_texto(lista_documento_entrada):  
    print(f">>> REALIZANDO A DIVISAO DO TEXXTO ORIGINAL EM CHUNKS")  
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)  
    documents = text_splitter.split_documents(lista_documento_entrada)  # usado split_documents dado que a entrada é uma lista de documentos:  
    i = 0  
    for pedaco in documents:  
        print("--" * 30)  
        print(f"Chunk: {i}")  
        print("--" * 30)  
        print(pedaco)  
        print("--" * 30)  
        i += 1  
    return documents  
  
  
# Cria o banco de dados vetorial, gerando os embeddings dos documentos  
def cria_banco_vetorial_e_indexa_documentos(documentos):  
    print(f">>> REALIZANDO INDEXAÇÃO DOS CHUNKS NO BANCO VETORIAL")  
    # Cria o banco de dados vetorial, gerando os embeddings dos documentos  
    # Adicionar os chunks no banco em lote    
    Chroma.from_documents(documentos, collection_name="nome_colecao", embedding=embeddings, persist_directory="./meu_banco")  
  
  
  
def ler_txt_e_retorna_texto_em_document():  
    print(f">>> REALIZANDO A LEITURA DO TXT EXEMPLO")  
    # lendo o txt com o texto exemplo e criando o Document:  
    lista_documentos = TextLoader('data/exemplo_texto.txt', encoding='utf-8').load()  
  
    print("Texto lido e convertido em Document")  
    print(lista_documentos)  
    print("-----------------------------------")  
    return lista_documentos  
  
  
def conecta_banco_vetorial_pre_criado():  
    vector_store_from_client = Chroma(  
        persist_directory="./meu_banco",  
        collection_name="nome_colecao",  
        embedding_function=embeddings,  
    )  
    return vector_store_from_client  
  
  
# Verifica se o diretório "./meu_banco" não existe  
if not os.path.exists("./meu_banco"):  
    print("O diretório './meu_banco' não existe... realizando a indexação")  
    texto_completo_lido = ler_txt_e_retorna_texto_em_document()  
    divide_texto = divide_texto(texto_completo_lido)  
    cria_banco_vetorial_e_indexa_documentos(divide_texto)  
else:  
    print("O diretório './meu_banco' já existe. Pulando a criação do banco vetorial.")  
  
# Conectando ao banco vetorial pre criado com os dados indexados:  
db = conecta_banco_vetorial_pre_criado()  
  
# Agora podemos trabalhar com o banco uma vez que ele está com os dados já indexados.  
  
query = "Na expansão da inteligência artificial quais questões importantes são levantadas?"  
pedacoes_retornados = db.similarity_search(query, k=2)  
  
  
# Total de docs retornados  
print("Total de pedaços. Deve ter o valor de 'K':")  
print(len(pedacoes_retornados))  
# Exibir o conteúdo do primeiro documento retornado  
# Imprimindo os pedaços retornados do banco:  
i=0  
for elm in pedacoes_retornados:  
    print(f"------ chunk {i} -------")  
    print(pedacoes_retornados[i].page_content)  
    print("--------------------")  
    i+=1